### Spark Session

In [1]:
import os
from pyspark import SparkConf
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *

# Adding AWS S3 Minio configs
sparkConf = (
    SparkConf()
    .set("spark.jars.ivy","/home/brijeshdhaker/.ivy2")
    .set("spark.ui.port", "4042")
    #.set("spark.jars.packages","org.apache.hadoop:hadoop-aws:3.0.0,io.delta:delta-spark_2.12:3.3.2")
    #.set("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
    #.set("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")
    #.set("spark.executor.heartbeatInterval", "300000")
    #.set("spark.network.timeout", "400000")
    # Minio specific settings
    #.set("spark.hadoop.fs.s3a.endpoint", "http://minio.sandbox.net:9010")
    #.set("spark.hadoop.fs.s3a.access.key", "pgm2H2bR7a5kMc5XCYdO")
    #.set("spark.hadoop.fs.s3a.secret.key", "zjd8T0hXFGtfemVQ6AH3yBAPASJNXNbVSx5iddqG")
    #.set("spark.hadoop.fs.s3a.path.style.access", "true")
    #.set("spark.hadoop.fs.s3a.aws.credentials.provider", "org.apache.hadoop.fs.s3a.SimpleAWSCredentialsProvider")
    #.set("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
    #.set("spark.sql.warehouse.dir", "s3a://defaultfs/spark/warehouse")
    #.set("spark.hadoop.fs.defaultFS", "s3a://defaultfs/")
    #.set("spark.eventLog.enabled", "true")
    #.set("spark.eventLog.dir", "file:///apps/var/logs/spark-events")
)

spark = (
    SparkSession.builder.master("local[*]").
        appName('spark-sql-notebook').
        config(conf=sparkConf).
        getOrCreate()
)

spark.sparkContext.setLogLevel('ERROR')
spark

25/10/16 06:49:07 WARN Utils: Your hostname, vmware-ubuntu-24.04 resolves to a loopback address: 127.0.1.1; using 192.168.154.133 instead (on interface ens33)
25/10/16 06:49:07 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/10/16 06:49:08 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
#
states = {"NY":"New York", "CA":"California", "FL":"Florida", "TX":"Texas", "CH":"Chicago", "OH":"Ohio", "NJ":"New Jersey", "UT":"Utah" }

#
#
#
sample_data = [
    (("James", "", "Smith"), "36636", "Sales", 34, "2006-01-01", "true", ["Java", "Scala", "C++"], {'hair': 'black', 'eye': 'brown'}, "M", 3100.50, "OH", "07-01-2019 12 01 19 406"),
    (("Michael", "Rose", ""), "40288", "Finance", 25, "2006-01-01", "true", ["Java", "Scala", "C++"], {'hair': 'brown', 'eye': 'brown'}, "M", 4300.65, "CA", "06-24-2019 12 01 19 406"),
    (("Robert", "", "Williams"), "42114", "Marketing", 27, "1992-06-23", "false", ["Java", "Scala", "C++"], {'hair': 'black', 'eye': 'black'}, "M", 1400.80, "NJ", "11-16-2019 16 50 59 406"),
    (("Maria", "Anne", "Jones"), "39192", "Finance", 19, "1987-05-05", "false", ["Java", "Scala", "C++"], {'hair': 'black', 'eye': 'brown'}, "F", 5500.75, "NY", "11-16-2019 16 50 59 406"),
    (("Jen", "Mary", "Brown"), "39191", "Marketing", 24, "1990-01-01", "false", ["Java", "Scala", "C++"], {'hair': 'black', 'eye': 'brown'}, "F", 0.00, "UT", "11-16-2019 16 50 59 406"),
    (("Jeff", "A", "Jones"), "39193", "Finance", 19, "2000-07-17", "false", ["Java", "Scala", "C++"], {'hair': 'black', 'eye': 'brown'}, "F", 5500.75, "CA", "11-16-2019 16 50 59 406"),
    (("Robert", "", "Williams"), "42114", "Marketing", 27, "1992-06-23", "false", ["Java", "Scala", "C++"], {'hair': 'black', 'eye': 'brown'}, "M", 1400.80, "NJ", "11-16-2019 16 50 59 406"),
    (("James", "", "Smith"), "36636", "Sales", 34, "2006-01-01", "true", ["Java", "Scala", "C++"], {'hair': 'black', 'eye': 'brown'}, "M", 3100.50, "OH", "07-01-2019 12 01 19 406"),
    (("Scott", "M", "Brown"), "39194", "Marketing", 24, "1994-10-01", "false", ["Python", "Scala", "C++"], {'hair': 'black', 'eye': 'brown'}, "F", 0.00, None, "11-16-2019 16 50 59 406")
]

#
#
#
sample_schema = StructType([
    StructField('name', StructType([
        StructField('firstname', StringType(), True),
        StructField('middlename', StringType(), True),
        StructField('lastname', StringType(), True)
    ])),
    StructField('id', StringType(), True),
    StructField('department', StringType(), True),
    StructField('age', IntegerType(), True),
    StructField('join_date', StringType(), True),
    StructField('isactive', StringType(), True),
    StructField('languages', ArrayType(StringType(), True), True),
    StructField('properties', MapType(StringType(),StringType()), True),
    StructField('gender', StringType(), True),
    StructField('salary', FloatType(), True),
    StructField('state', StringType(), True),
    StructField('timestamp', StringType(), True)
])

In [3]:
#
df = spark.createDataFrame(data = sample_data, schema = sample_schema)

#
df.printSchema()

#
df.show(truncate=False)

root
 |-- name: struct (nullable = true)
 |    |-- firstname: string (nullable = true)
 |    |-- middlename: string (nullable = true)
 |    |-- lastname: string (nullable = true)
 |-- id: string (nullable = true)
 |-- department: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- join_date: string (nullable = true)
 |-- isactive: string (nullable = true)
 |-- languages: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- properties: map (nullable = true)
 |    |-- key: string
 |    |-- value: string (valueContainsNull = true)
 |-- gender: string (nullable = true)
 |-- salary: float (nullable = true)
 |-- state: string (nullable = true)
 |-- timestamp: string (nullable = true)



+--------------------+-----+----------+---+----------+--------+--------------------+-----------------------------+------+-------+-----+-----------------------+
|name                |id   |department|age|join_date |isactive|languages           |properties                   |gender|salary |state|timestamp              |
+--------------------+-----+----------+---+----------+--------+--------------------+-----------------------------+------+-------+-----+-----------------------+
|{James, , Smith}    |36636|Sales     |34 |2006-01-01|true    |[Java, Scala, C++]  |{eye -> brown, hair -> black}|M     |3100.5 |OH   |07-01-2019 12 01 19 406|
|{Michael, Rose, }   |40288|Finance   |25 |2006-01-01|true    |[Java, Scala, C++]  |{eye -> brown, hair -> brown}|M     |4300.65|CA   |06-24-2019 12 01 19 406|
|{Robert, , Williams}|42114|Marketing |27 |1992-06-23|false   |[Java, Scala, C++]  |{eye -> black, hair -> black}|M     |1400.8 |NJ   |11-16-2019 16 50 59 406|
|{Maria, Anne, Jones}|39192|Finance   |1

#### Sorting Rows

In [9]:
df.select("id","department").sort(desc("id")).show()

+-----+----------+
|   id|department|
+-----+----------+
|42114| Marketing|
|42114| Marketing|
|40288|   Finance|
|39194| Marketing|
|39193|   Finance|
|39192|   Finance|
|39191| Marketing|
|36636|     Sales|
|36636|     Sales|
+-----+----------+



In [10]:
df.select("id","department").sort(asc("id")).show()

+-----+----------+
|   id|department|
+-----+----------+
|36636|     Sales|
|36636|     Sales|
|39191| Marketing|
|39192|   Finance|
|39193|   Finance|
|39194| Marketing|
|40288|   Finance|
|42114| Marketing|
|42114| Marketing|
+-----+----------+



In [13]:
df.select("id","department").distinct().show()

+-----+----------+
|   id|department|
+-----+----------+
|36636|     Sales|
|40288|   Finance|
|42114| Marketing|
|39192|   Finance|
|39191| Marketing|
|39193|   Finance|
|39194| Marketing|
+-----+----------+



#### distinct rows

In [10]:
df.select("department").distinct().show()

+----------+
|department|
+----------+
|     Sales|
|   Finance|
| Marketing|
+----------+



#### Date Columns

In [ ]:
#
from pyspark.sql.functions import to_date, to_timestamp

# Convert string to date
df.select("name.*", to_date("join_date", "yyyy-MM-dd").alias("start_date")).show(truncate=False)

# Cast string to date
df.withColumn("date_col", col("join_date").cast("date")).show(truncate=False)

In [ ]:
# Convert string to date
df.withColumn("date_col", 
from_unixtime(unix_timestamp("join_date", "yyyy-MM-dd")).cast("date")).show(truncate=False)

In [ ]:
from pyspark.sql.functions import date_format
# Convert string to date
df = df.withColumn("formatted_date", 
date_format("join_date", "yyyy-MM-dd")).show(truncate=False)

#### Timestamp

In [ ]:
# Convert the 'timestamp' column to a timestamp type
# Specify the format of the input string
df_converted = df.withColumn("add_ts", to_timestamp(col("timestamp"), "MM-dd-yyyy HH mm ss SSS"))

# Show the updated DataFrame and its schema to verify the data type
df_converted.show(truncate=False)

# Cast string to timestamp
df.withColumn("timestamp_col", col("timestamp").cast("timestamp")).show(truncate=False)

In [ ]:
from pyspark.sql.functions import unix_timestamp, from_unixtime
# Convert string to timestamp
df.withColumn("timestamp_col", 
from_unixtime(unix_timestamp("timestamp", "MM-dd-yyyy HH mm ss SSS"))).show(truncate=False)


#### Select Nested columns

In [ ]:
# Select columns from struct type
df.select("name.firstname","name.middlename","name.lastname").show(truncate=False)


In [ ]:
# Extract all columns from struct type
df.select("name.*").show(truncate=False)

#### Explode Arry Type columns 

In [ ]:
#
df.select("languages", explode(col("languages")).alias("speak_language")).show(truncate=False)

In [ ]:
#
df.withColumn("known_language", explode(col("languages"))).show(truncate=False)

#### Access PySpark MapType Elements

In [ ]:
#
df3=df.rdd.map(lambda x: (x.name, x.properties["hair"], x.properties["eye"])).toDF(["name","hair","eye"])
df3.printSchema()
df3.show()

In [ ]:
df.withColumn("hair",df.properties.getItem("hair")) \
  .withColumn("eye",df.properties.getItem("eye")) \
  .drop("properties") \
  .show()

df.withColumn("hair",df.properties["hair"]) \
  .withColumn("eye",df.properties["eye"]) \
  .drop("properties") \
  .show()

In [ ]:
#
df.select(df.name,explode(df.properties)).show()

In [ ]:
# map_keys() – Get All Map Keys
df.select(df.name,map_keys(df.properties)).show()

In [ ]:
# map_values() – Get All map Values
from pyspark.sql.functions import map_values
df.select(df.name,map_values(df.properties)).show()


### Spark Configurations

#### Set

In [ ]:
# Disable AQE and Broadcast join

spark.conf.set("spark.sql.adaptive.enabled", False)
spark.conf.set("spark.sql.adaptive.coalescePartitions.enabled", False)
spark.conf.set("spark.sql.autoBroadcastJoinThreshold", -1)

In [ ]:
spark.conf.set("spark.sql.shuffle.partitions", "8")

#### Get

In [ ]:
print(spark.conf.get("spark.sql.shuffle.partitions"))

# Check Spark defaultParallelism
print(spark.sparkContext.defaultParallelism)

### Broadcast Variables

In [ ]:
#
broadcastStates = spark.sparkContext.broadcast(states)
# print("{}".format(broadcastStates.value))

#
def state_convert(code):
    state_name = ""
    if code is not None and code != "" : 
      state_name = broadcastStates.value[code] 
    return state_name

#
result = df.rdd.map(lambda x: (x[0],x[1],x[2],state_convert(x[10]))).toDF(["name","id","department","state_fullname"])
result.show(truncate=False)

## Broadcast variable on filter
filteDf= df.where((df['state'].isin(list(broadcastStates.value.keys()))))
filteDf.show(truncate=False)

### Accumulators Variables

In [ ]:
# Create Accumulator
sc_acc = spark.sparkContext.accumulator(0)
print("Accumulator initial value: {}".format(sc_acc.value))

#
data = range(1, 101)
rdd_1 = spark.sparkContext.parallelize(data, 8)
print("RDD-1 Partition Count : %i " % (rdd_1.getNumPartitions()))
print("Values in RDD-1 : {0} ".format(rdd_1.collect()))

#
# commons.print_separator()

# Accessed By Driver
rdd_1.foreach(lambda x: sc_acc.add(x))
print("Total Sum  {0} ".format(sc_acc.value))

### Read Data

In [ ]:
## By default, Spark's CSV reader treats empty strings ("") and blank values (e.g., ,, where nothing is between the commas) as null

# This code will read "N/A" as null
dfFromCSV = spark.read \
    .option("header", True) \
    .option("delimiter", ',') \
    .option("emptyValue", 'N/A') \
    .option("nullValue", 0) \
    .option("treatEmptyValuesAsNulls", True) \
    .options(inferSchema=True, delimiter=',') \
    .csv("file:///apps/sandbox/defaultfs/employee.csv")

#dfFromCSV.printSchema()
dfFromCSV.show(truncate=False)


In [ ]:
dfFromJSON = spark.read.format("json").load("file:///apps/sandbox/defaultfs/cdrs.json")
dfFromJSON.printSchema()
dfFromJSON.show(truncate=False)

In [ ]:
dfParquet = spark.read.format("parquet").load("file:///apps/sandbox/defaultfs/taxi-data/yellow_tripdata_2021-04.parquet")
dfParquet.printSchema()
dfParquet.show(truncate=False)

In [ ]:
employee_schema = StructType() \
    .add("emp_id", IntegerType(), True) \
    .add("emp_name", StringType(), True) \
    .add("emp_role", StringType(), True) \
    .add("emp_manager", StringType(), True) \
    .add("emp_hiredate", DateType(), True) \
    .add("emp_salary", IntegerType(), True) \
    .add("emp_comm", IntegerType(), True) \
    .add("emp_dept", IntegerType(), True)

df_with_schema = spark.read.format("csv") \
    .option("header", True) \
    .schema(employee_schema) \
    .load("file:///apps/sandbox/defaultfs/employee.csv")

#df_with_schema.printSchema()

df_with_schema.show(truncate=False)

### Handeling Null & Empyty Values

In [ ]:
#
## Replacing null values with some value
#
df_with_schema.fillna(value=0).show()
df_with_schema.fillna(value=0,subset=["emp_comm"]).show() # only for emp_comm column
df_with_schema.fillna(10,["emp_comm"]) \
    .fillna("---",["emp_manager"]).show()

In [ ]:
#
## Replacing null values with some value using na
#
df_with_schema.na.fill(value=0).show()
df_with_schema.na.fill(value=0,subset=["emp_comm"]).show()

In [ ]:

employee_columns = ['emp_id', 'emp_name', 'emp_role', 'emp_manager', 'emp_hiredate', 'emp_salary', 'emp_comm', 'emp_dept']

employee_schema = StructType() \
    .add("emp_id", IntegerType(), True) \
    .add("emp_name", StringType(), True) \
    .add("emp_role", StringType(), True) \
    .add("emp_manager", StringType(), True) \
    .add("emp_hiredate", DateType(), True) \
    .add("emp_salary", IntegerType(), True) \
    .add("emp_comm", IntegerType(), True) \
    .add("emp_dept", IntegerType(), True)

employee_df = spark.read.csv("file:///apps/sandbox/defaultfs/employee.csv",
    header=True,
    schema=employee_schema
)

#
# employee_df.printSchema()

#
# print(employee_df.rdd.getNumPartitions())

#
employee_df.show(truncate=False)

In [ ]:

dept_columns = ['dept_id', 'dept_name', 'dept_location']

dept_schema = StructType() \
    .add("dept_id", IntegerType(), True) \
    .add("dept_name", StringType(), True) \
    .add("dept_location", StringType(), True)

dept_df = spark.read.format("csv") \
    .option("header", True) \
    .schema(dept_schema) \
    .load("file:///apps/sandbox/defaultfs/departments.csv")

#dept_df.printSchema()

dept_df.show(truncate=False)

### Data Repartition

In [ ]:
#
employee_df = employee_df.repartition(2)

#
print(employee_df.rdd.getNumPartitions())

#
# employee_df.show(truncate=False)

## Process Dataframe

### Map Dataframe Partitions

In [ ]:

def applyMap(row):
    bonus = 0
    salary = 0

    if row.emp_comm is None :
        bonus = 0
    else :
        bonus = row.emp_comm


    if row.emp_role == 'ANALYST':
        salary = row.emp_salary + 1000
        bonus = bonus + 100
    elif row.emp_role == 'CLERK':
        salary = row.emp_salary + 1500
        bonus = bonus + 150
    elif row.emp_role == 'MANAGER':
        salary = row.emp_salary + 2000
        bonus = bonus + 200
    elif row.emp_role == 'SALESMAN':
        salary = row.emp_salary + 2500
        bonus = bonus + 250
    else:
        salary = row.emp_salary
        bonus = 0
    
    return (row.emp_id, row.emp_name, row.emp_role, row.emp_manager, row.emp_hiredate, salary, bonus, row.emp_dept)


df1_columns = ["emp_id","emp_name", "emp_role", "emp_manager", "emp_hiredate", "emp_salary", "emp_comm", "emp_dept"]

##
#df1 = employee_df.rdd.map(lambda r: (r.emp_id, r.emp_name, r.emp_role, r.emp_manager, r.emp_hiredate, r.emp_salary, 225, r.emp_dept)).toDF(df1_columns)

##
df1 = employee_df.rdd.map(applyMap).toDF(df1_columns)
df1.show(truncate=False)


### flatMap Dataframe Partitions

### Process Dataframe Partitions
Unfortunately, PySpark DataFame doesn’t have flatMap() transformation however, DataFrame has explode() SQL function that is used to flatten the column. Below is a complete example.

In [ ]:
arrayData = [
        ('James',['Java','Scala'],{'hair':'black','eye':'brown'}),
        ('Michael',['Spark','Java',None],{'hair':'brown','eye':None}),
        ('Robert',['CSharp',''],{'hair':'red','eye':''}),
        ('Washington',None,None),
        ('Jefferson',['1','2'],{})]
df = spark.createDataFrame(data=arrayData, schema = ['name','knownLanguages','properties'])

from pyspark.sql.functions import explode
df2 = df.select(df.name, explode(df.knownLanguages))
df2.printSchema()
df2.show()

In [ ]:
# Sample DataFrame
data = [("Spark is great",), ("Map and FlatMap are useful",)]
df = spark.createDataFrame(data, ["sentence"])

# Use 'explode' function to achieve flatMap-like behavior
df_words = df.withColumn("words", explode(split(col("sentence"), " ")))

df_words.show()

In [ ]:
# Using mapPartitions with yield
def formatWithYield(partition_data):
    for record in partition_data:
        role = 'ANALYST' if  record.emp_role == 'CLERK' else 'MANAGER'
        salary = record.emp_salary
        if record.emp_salary <= 1000 :
            salary = record.emp_salary * 10
        
        bonus = record.emp_comm 
        if record.emp_comm is None :
            bonus = 50
        yield (record.emp_id, record.emp_name, record.emp_role, record.emp_manager, record.emp_hiredate, salary, bonus, record.emp_dept)




df2_columns = ["emp_id","emp_name", "emp_role", "emp_manager", "emp_hiredate", "emp_salary", "emp_comm", "emp_dept"]
df2 = employee_df.rdd.mapPartitions(formatWithYield).toDF(df2_columns)
df2.show(truncate=False)



In [ ]:
# Using mapPartitions with iterator
def formatWithIter(partition_data):
    p_data = []
    for record in partition_data:
        role = 'ANALYST' if  record.emp_role == 'CLERK' else 'MANAGER'
        salary = record.emp_salary
        if record.emp_salary <= 1000 :
            salary = record.emp_salary * 10
        
        bonus = record.emp_comm 
        if record.emp_comm is None:
            bonus = 50
        
        p_data.append([record.emp_id, record.emp_name, record.emp_role, record.emp_manager, record.emp_hiredate, salary, bonus, record.emp_dept])
    return iter(p_data)

df3_columns = ["emp_id","emp_name", "emp_role", "emp_manager", "emp_hiredate", "emp_salary", "emp_comm", "emp_dept"]
df3 = employee_df.rdd.mapPartitions(formatWithIter).toDF(df3_columns)
df3.show(truncate=False)

### Process Dataframe with mapPartitionsWithIndex

In [ ]:
#
#
#
def formatWithMapPartitionsWithIndex(partitionIndex, iterator):
    for paartion_data in iterator:
        role = 'ANALYST' if  paartion_data.emp_role == 'CLERK' else 'MANAGER'
        salary = paartion_data.emp_salary
        if paartion_data.emp_salary <= 1000 :
            salary = paartion_data.emp_salary * 10
        
        bonus = paartion_data.emp_comm 
        if paartion_data.emp_comm is None:
            bonus = 50
        
        yield (partitionIndex+1, paartion_data.emp_id, paartion_data.emp_name, role, paartion_data.emp_manager, paartion_data.emp_hiredate, salary, bonus, paartion_data.emp_dept)
    
    #yield (partitionIndex, len(list(iterator)))
    

df4_columns = ["partition","emp_id","emp_name", "emp_role", "emp_manager", "emp_hiredate", "emp_salary", "emp_comm", "emp_dept"]
df4 = employee_df.rdd.mapPartitionsWithIndex(formatWithMapPartitionsWithIndex).toDF(df4_columns)
df4.show(truncate=False)

In [ ]:
employee_df = employee_df.filter(col("emp_salary") > 2000)\
    .select("emp_id", "emp_name", "emp_dept", "emp_salary")\
    .groupby("emp_dept")\
    .count()

In [ ]:
employee_df.collect()

In [ ]:
df = spark.range(1,6)
df.show()

In [ ]:
df.select("id").show()

In [ ]:
df.select((df.id).alias("##"), col("id").alias("#ID") ,(df.id + 10).alias('###COL')).show()

In [ ]:
df.selectExpr("id * 5" , "id").show()

In [ ]:
names = spark.createDataFrame(data=[(1000, 'Nick'), (1001, 'John'), (1002, 'Frank')], schema=['id', 'name'])
names.printSchema()

In [ ]:
names.select('id', 'name').show()

#### Q-001. If this value is set to a number other than 200, the number of tasks in the shuffle stage will reflect that new value.

`Adaptive Query Execution (AQE)`: In Spark 3.0 and later, Adaptive Query Execution (AQE) is an optimization that can dynamically adjust the number of shuffle partitions during runtime. AQE can coalesce small shuffle partitions into larger ones, effectively reducing the number of tasks if the data distribution allows for it. If AQE is enabled and determines that fewer partitions are optimal, you will see fewer than 200 tasks.

`Data Volume and Distribution`: If the amount of data being processed is very small, or if the data is highly skewed (meaning a few keys have a disproportionately large amount of data), Spark might not utilize all 200 partitions efficiently, or AQE might optimize the partition count.

`Coalesce or Repartition before GroupBy`: If a coalesce or repartition operation was performed on the DataFrame immediately before the groupBy, it could explicitly set the number of partitions, overriding the default shuffle partition setting for that specific operation.

`Specific Optimization Strategies`: Certain optimization strategies or custom partitioning schemes might be in place that influence the number of partitions used during the groupBy operation, leading to a task count different from 200.

In [ ]:
names.select(col("id")).show()

In [ ]:
from pyspark.sql import SparkSession

# Spark session & context
spark = SparkSession.builder.master("local").getOrCreate()
sc = spark.sparkContext

# Sum of the first 100 whole numbers
rdd = sc.parallelize(range(100 + 1))
rdd.sum()
# 5050

In [ ]:
##
## spark.read.format("csv").option("header", "true").load("file:///apps/sandbox/defaultfs/employee.csv")
## Load data from csv
##

employee_df = spark.read.csv("file:///apps/sandbox/defaultfs/employee.csv",
    header=True,
    nullValue="NA",
    inferSchema=True
)

#employee_df.printSchema()
print(employee_df.rdd.getNumPartitions())

employee_df = employee_df.repartition(2)
print(employee_df.rdd.getNumPartitions())

employee_df = employee_df.filter(col("emp_salary") > 2000)\
    .select("emp_id", "emp_name", "emp_dept", "emp_salary")\
    .groupby("emp_dept")\
    .count()


employee_df.collect()

#### Get highest salary of each group  

In [ ]:
#Get highest salary of each group  
w3 = Window.partitionBy("department").orderBy(col("salary").desc())
df.withColumn("row",row_number().over(w3)) \
  .filter(col("row") == 1).drop("row") \
  .show()

#### Get max, min, avg, sum of each group

In [ ]:
w4 = Window.partitionBy("department")
df.withColumn("row",row_number().over(w3)) \
  .withColumn("avg", avg(col("salary")).over(w4)) \
  .withColumn("sum", sum(col("salary")).over(w4)) \
  .withColumn("min", min(col("salary")).over(w4)) \
  .withColumn("max", max(col("salary")).over(w4)) \
  .where(col("row")==1).select("department","avg","sum","min","max") \
  .show()

In [ ]:
data = [
        Row(name="James,,Smith", lang=["Java", "Scala", "C++"], state="CA"),
        Row(name="Michael,Rose,", lang=["Spark", "Java", "C++"], state="NJ"),
        Row(name="Robert,,Williams", lang=["CSharp", "VB"], state="NV")
]

#DataFrame Example 1
columns = ["name", "languagesAtSchool", "currentState"]
df1 = spark.createDataFrame(data)
df1.printSchema()
df1.show()

#DataFrame Example 2

columns = ["name", "languagesAtSchool", "currentState"]
df2 = spark.createDataFrame(data).toDF(*columns)
df2.printSchema()
df2.show()

In [ ]:
structureData = [
    (("James", "", "Smith"), "36636", "M", 3100),
    (("Michael", "Rose", ""), "40288", "M", 4300),
    (("Robert", "", "Williams"), "42114", "M", 1400),
    (("Brijesh", "K", "Dhaker"), "400706", "M", 3000),
    (("Maria", "Anne", "Jones"), "39192", "F", 5500),
    (("Jen", "Mary", "Brown"), "", "F", -1)
]
structureSchema = StructType([
    StructField('name', StructType([
        StructField('firstname', StringType(), True),
        StructField('middlename', StringType(), True),
        StructField('lastname', StringType(), True)
    ])),
    StructField('id', StringType(), True),
    StructField('gender', StringType(), True),
    StructField('salary', IntegerType(), True)
])

df2 = spark.createDataFrame(data=structureData, schema=structureSchema)
df2.printSchema()
df2.show(truncate=False)

from pyspark.sql.functions import col,struct,when
#
#
#
updatedDF = df2.withColumn("OtherInfo",
                           struct(col("id").alias("identifier"),
                                  col("gender").alias("gender"),
                                  col("salary").alias("salary"),
                                  when(col("salary").cast(IntegerType()) < 2000, "Low")
                                  .when(col("salary").cast(IntegerType()) < 4000, "Medium")
                                  .otherwise("High").alias("Salary_Grade")
                           )).drop("id", "gender", "salary")

updatedDF.printSchema()
updatedDF.show(truncate=False)